In [1]:
import torch
from utils.experimentorUtils import getExperimentor
from utils.constants import *
print(torch.backends.cudnn.version())

8005


In [2]:
config = {
    "dataset_name": Dataset.OGBN_ARXIV,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 500,
    'num_of_runs': 1,
    'patience_period': 100,
    
    'batch_size': 256,
    'test_batch_size': 256,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 5,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.005,
    'num_of_layers': 2, 
    'num_heads': 8,
    'hidden_size': 128,
    'dropout': 0.6,  
    "use_layer_norm": False,
    "use_batch_norm": False,
    
    'nbor_degree': 1,
    'adj_mode': AdjacencyMode.OneStep,
    'sparse': False
}

In [3]:
experimentor = getExperimentor(config["dataset_name"])(config)
experimentor.run_wrapper()

Run 01:


Epoch 01: 100%|██████████| 90941/90941 [00:20<00:00, 4346.09it/s] 


Epoch 01| Loss: 1.8062| Acc: 0.5093| Train Time: 20.9287s
